<a href="https://colab.research.google.com/github/daiyon-cho/DualStyleGAN/blob/main/0_merge_filter_member_action_pref_styles_py_version_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import some libraries

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import requests
import zipfile
import os
import ast
import json
from dateutil.parser import parse
from IPython.display import Image
import IPython.display as display
from datetime import date
from dateutil.parser import parse
from collections import Counter


#-------remove wrong age and make age code--------------------------------------

def remove_wrong_age_and_make_agecode(members):
  # member의 나이를 계산하여 members data에 age 컬럼 추가
  # 결측치가 있는 경우 nan으로 표시
  members['age'] = members['birthday'].apply(lambda x: calculate_age(parse(x)) if pd.notnull(x) else np.nan)

  # 나이가 잘못된 데이터 제거
  # setting with copy warning 문제로 새로운 new_members_df 리스트를 만든 후 새로 컬럼을 추가함
  new_members = members.loc[(members['age'] > 10) & (members['age'] < 60)].copy()

  # age는 추후 review한 멤버들의 평균 나이를 계산한 결과로 바뀔 것임
  # o_age는 멤버가 입력한 생년월일로 계산한 멤버의 본래 나이
  # o_age의 age code를 생성하여 새 컬럼으로 추가
  new_members['o_age'] = new_members['age']
  new_members['o_agecode'] = new_members['age'].apply(lambda x: generate_agecode_with_age(x) if pd.notnull(x) else np.nan)

  return new_members


#------------get style data only-----------------------------------------------

def get_style_data_only(df):
  # 스타일 이미지 관련된 data만 선별
  style_df = df[df['refType']=='STYLE']  # style 만 선정
  return style_df


#------------나이 계산 함수 -----------------------------------------------
# calculate age from birthdate
# usage example: calculate_age(parse(members['birthday'][50]))
# 사용 예: calculate_age(parse(members['birthday'][11100]))

def calculate_age(birthdate):
    today = date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age


#-----------나이로 나이 코드를 생성하는 함수-----------------------------------
def generate_agecode_with_age(age):
  if (age > 9) & (age < 20):
    age_code = '643a3702375c000078005c82'
  elif (age >= 20) & (age < 30):
    age_code = '643a370c375c000078005c83'
  elif (age >= 30):
    age_code = '643a3718375c000078005c84'
  return age_code

#---------------style-----------------------------------------------------------
with open('codes.json', 'r', encoding="UTF-8") as f:
  data = json.load(f)

#-------------style dictionary---------------------

# 남 스타일 사전. Create a dictionary with "title", "en" as key and "_id" as value for men style
m_style_dict = {item['title']['en']: item['_id'] for item in data['data']['styleCatalog']['men']}
# 여 스타일 사전. Create a dictionary with "title", "en" as key and "_id" as value for women style
w_style_dict = {item['title']['en']: item['_id'] for item in data['data']['styleCatalog']['women']}

# 남녀 모두의 통합 스타일 사전
style_dict = {**m_style_dict, **w_style_dict}

# reversed category dictionary 생성
style_dict_r = {v: k for k, v in style_dict.items()}


#----------list agg-------------------------------------------------------------
# aggreate 수행시 string을 모아서 list를 만들기 위해 만든 함수

def list_agg(x):
  return list(x)


#-------------count actions for a style-----------------------------------------
# 각 멤버 별 action 수 집계하여 새 컬럼 n_actions 생성

def count_actions_for_style(df):
  df['n_actions'] = df['refId'].apply(len)
  return df


#------------get liked style using member's action------------------------------
# member가 등록 시 선호한 이미지들의 스타일을 styles 데이터로부터 알아내는 함수
# mem_act_pref_df : members, actions merge data를 다시 member 중심으로 action들을 aggregate한 dataframe
# st_df : styles dataframe
# st_dict_r : style code 입력하여 style title을 찾는 dictionary
# nth : mem_act_pref_df에서 n번째 member
# output sorted list, percent list

def get_liked_style_using_members_action(m_a_pref, st_df, st_dic_r, nth):
  imgId_lst = m_a_pref['refId'][nth]       # nth member가 선호한 style image들의 id list
  title_lst = []
  for imgId in imgId_lst:
    st_image = st_df.loc[st_df['_id'] == imgId]       # 해당 style image 추출
    if not st_image.empty:                            # 해당 style image가 styles에 있는 경우만 다음을 실행
      st_code = st_image['attStyle[0]'].values[0]     # 해당 style image의 style code를 추출
      title_lst.append(st_dic_r[st_code])             # style title을 추출하여 리스트에 추가
  srt_lst = sorted(list(Counter(title_lst).items()), key=lambda x: x[1], reverse=True)   # style 별 frequency를 구하고 sort
  total = sum([i[1] for i in srt_lst])                # style 별 frequency의 총합을 구함
  pct_lst = [(i[0], round(i[1]/total*100, 2)) for i in srt_lst]   # style별 %를 구함
  return srt_lst, pct_lst


#---------- reading data --------------------------------------------------
styles0 = pd.read_csv('showniq_db.STYLE_MAIN.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="UTF-8")
members0 = pd.read_csv('showniq_db.MEMBER_MAIN.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="UTF-8")
actions0 = pd.read_csv('showniq_db.MEMBER_ACTION_MAIN.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="UTF-8")
reviews0 = pd.read_csv('showniq_db.REVIEW_MAIN.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="UTF-8")


#----filtering: select needed columns from every database ------------------
# select needed columns from members0
members = members0[['_id', 'birthday', 'gender', 'genderFilter', 'createdAt']]

# select needed columns from styles0
styles = styles0[['_id', 'image[0].thumb.url', 'url',
                  'likedCnt', 'reviewCnt', 'sharedCnt',
                  'attGender[0]', 'attStyle[0]', 'attAge[0]', 'attAge[1]',
                  'reviewStatistic.rating',
                  'reviewStatistic.detail[0].ageId', 'reviewStatistic.detail[0].value',
                  'createdAt']]

# select needed columns from reviews0
reviews = reviews0[['_id', 'refType', 'refId', 'emoType', 'likedCnt', 'createdId']]

# select needed columns from actions0
actions = actions0[['_id', 'actionType', 'memberId', 'refId', 'refType', 'gender', 'memberType']]


#------ merging and aggregating data----------------------------------------------
##-------------- actions-members merge, aggregate, and save ---------------------

members_filtered = remove_wrong_age_and_make_agecode(members)
# actions에서 'refId'가 styles의 '_id'에 있는지 확인하여 있는 것들만 선별함
actions = actions[actions['refId'].isin(styles['_id'])]
actions_filtered = get_style_data_only(actions)

# STYLE 선별과 동일하나 결과는 약간 차이가 있음
actions_st_liked = actions_filtered[actions_filtered['actionType']=='LIKED'] # style 만 likded 하기 때문

# 멤버가 등록 시 선택한 이미지를 찾기 위한 merge임
# action을 취한 멤버의 id를 member id와 일치하도록 merge를 행하여
# 각 멤버가 몇 개의 스타일 이미지에 대하여 action을 취했는지
# 한 멤버들의 수와 나이 평균을 구함
# how = 'inner'는 멤버만을 대상으로 하며
# how = 'outer'는 멤버와 게스트를 모두 포함하게 됨
# GUEST 의 경우 memberId가 없으면 추천 로직 상 사용될 수 없기 때문에 이 과정은 사용하지 않음
actions_members = actions_st_liked.merge(members_filtered, left_on = 'memberId', right_on = '_id', how = 'inner')

## actions-members aggregate by member
# 한 멤버가 여러 개의 action을 통하여 선택한 여러 개의 스타일 이미지를 리스트로 만듦
actions_members_pref = actions_members.groupby(['memberId']).agg({'_id_x' : 'first',         # beginning of actions
                                                                  'actionType' : 'first',
                                                                  'memberId' : 'first',
                                                                  'refId' : list_agg,  # 한 멤버가 몇 개의 스타일을 선택했는지
                                                                  'refType' : 'first',
                                                                  'gender_x' : 'first',
                                                                  'memberType' : 'first',
                                                                  '_id_y' : 'first',         # beginning of members
                                                                  'birthday' : 'first',
                                                                  'gender_y' : 'first',
                                                                  'genderFilter' : 'first',
                                                                  'createdAt' : 'first',
                                                                  'age' : 'first',
                                                                  'o_age' : 'first',
                                                                  'o_agecode' : list_agg  # 동일한 멤버의 연령. 의미 없음
                                                                  })

actions_members_pref = count_actions_for_style(actions_members_pref)

# members_acionts_pref의 모든 멤버에게 함수를 적용시켜 각 멤버의 스타일 선호를 계산함 ... 3시간 걸림
sslist = []
pplist = []
for i in range(len(actions_members_pref)):
  print('i = ', i)
  slist, plist = get_liked_style_using_members_action(actions_members_pref, styles, style_dict_r, i)
  sslist.append(slist)
  pplist.append(plist)

# 함수를 적용해 생성한 sorted list, percent list를 actions_members_pref에 새로운 컬럼으로 생성
actions_members_pref['pref_st_sort'] = sslist
actions_members_pref['pref_st_perc'] = pplist

# actions_members_pref save
# dataframe을 csv file로 저장
actions_members_pref.to_csv('actions_members_pref.csv', index=False)


##-------------reviews styles merge, aggregate ---------------------------------
reviews_st = reviews[reviews['refType'] == 'STYLE']

# reviews와 styles의 merge
# 스타일 id 위주. refId가 스타일 이미지 id.
# 이 정보를 근거로 스타일 이미지가 부여받은 평점들의 평균을 계산함
# merge로 avg_rating 컬럼이 새로 생겼음. 모두 값이 NaN으로 되어있음
reviews_styles = reviews_st.merge(styles, left_on = 'refId', right_on = '_id', how = 'inner')

# style을 중심으로 여러 review를 종합함. 한 style image에 대한 reviewer들의 rating 평균을 구함
reviews_styles_rating_mean = reviews_styles.groupby(['refId']).agg({'_id_x' : 'first',       # beginning of reviews
                                                                    'refType' : 'first',
                                                                    'refId' : 'first',
                                                                    'emoType' : 'mean',      # average rating of reviewers to a style image
                                                                    'likedCnt_x' : 'first',
                                                                    'createdId' : 'first',
                                                                    '_id_y' : 'first',       # beginning of styles
                                                                    'image[0].thumb.url' : 'first',
                                                                    'url' : 'first',
                                                                    'likedCnt_y' : 'first',
                                                                    'reviewCnt' : 'first',
                                                                    'sharedCnt' : 'first',
                                                                    'attGender[0]' : 'first',
                                                                    'attStyle[0]' : 'first',
                                                                    'attAge[0]' : 'first',
                                                                    'attAge[1]' : 'first',
                                                                    'reviewStatistic.rating' : 'first',
                                                                    'reviewStatistic.detail[0].ageId' : 'first',
                                                                    'reviewStatistic.detail[0].value' : 'first',
                                                                    'createdAt' : 'first'
                                                                    })


##-------------reviews members merge and caculate mean age ---------------------
# style image를 review한 member들의 평균 연령을 계산하기 위함
# [reviews | createdId] : 스타일 이미지를 리뷰 한 멤버의 id
# [members | _id] : 멤버의 id
reviews_members = reviews_st.merge(members_filtered, left_on = 'createdId', right_on = '_id', how = 'inner')

# review를 중심으로 review한 member들의 정보를 모음.
# member들이 특정 refId(Style image)를 review한 rating의 평균과 평균 나이를 구함
reviews_members_mean = reviews_members.groupby(['refId']).agg({'_id_x' : 'first',   # beginning of reviews
                                        'refType' : 'first',
                                        'refId' : 'first',
                                        'emoType' : 'mean',                         # average rating to many style images of one member
                                        'likedCnt' : 'first',
                                        'createdId' : 'first',
                                        '_id_y'  : 'first',                         # beginning of members
                                        'birthday' : 'first',
                                        'gender' : 'first',
                                        'genderFilter' : 'first',
                                        'createdAt' : 'first',
                                        'age' : 'mean',                             # average age of reviewers to a style image
                                        'o_age' : 'first',
                                        'o_agecode' : 'first',
                                        })

#------- merge reviews_members_mean to reviews_styles_rating_mean and save --------------------
# 두 dataframe 이 모두 'refId' 컬럼을 index로 사용하고 있어서, 이들 index를 제거하기 위해 reset함
# 그 후 merge를 해야 함
reviews_styles_rating_mean.reset_index(drop=True, inplace=True)
reviews_members_mean.reset_index(drop=True, inplace=True)

# merge할 경우 둘 다 'refId'를 사용하므로 왼쪽 dataframe의 row가 더 많아야 함
reviews_styles_rating_mean = reviews_styles_rating_mean.merge(reviews_members_mean, left_on='refId', right_on='refId', how='inner')

# dataframe을 csv file로 저장
reviews_styles_rating_mean.to_csv('reviews_styles_rating_mean.csv', index=False)
